In [1]:
##for a pair of odds where x and y are complements of each other i.e. duke winning vs UNC or UNC winning vs Duke
def odds_to_prob2(x, y):
    if (x < 0):
        prob_x = (-1 * x) / (100 - x)
    else:
        prob_x = 100 / (x + 100)
    if (y < 0):
        prob_y = (-1 * y) / (100 - y)
    else:
        prob_y = 100 / (y + 100)
    vig = prob_x + prob_y - 1.0
    prob_x -= vig/2
    prob_y -= vig/2
    return (prob_x, prob_y)

def vig(x, y):
    if (x < 0):
        prob_x = (-1 * x) / (100 - x)
    else:
        prob_x = 100 / (x + 100)
    if (y < 0):
        prob_y = (-1 * y) / (100 - y)
    else:
        prob_y = 100 / (y + 100)
    vig = prob_x + prob_y - 1.0
    return vig

##Converting 
def odds_to_prob1(odds):
    if (odds < 0):
        return (-1 * odds) / (100 - odds)
    else:
        return 100 / (odds + 100)

##exp value payout for a bet size of $1
def calc_statistical_edge(odds, consensus_prob):
    if (odds < 0):
        payout = ((-100 / odds) + 1)
    else:
        payout = ((odds / 100) + 1)
    return payout * consensus_prob


def exp_value(odds, consensus_prob, principle):
    if (odds < 0):
        payout = ((-100 / odds) + 1) * principle
    else:
        payout = ((odds / 100) + 1) * principle
    return payout * consensus_prob

##If you want the statistical edge, just do the expected value for a principle of $1

def consensus_prob_calc(probs, weights=None):
    if not weights: # fallback value
        weights = [1/len(probs)] * len(probs)
    weights = [weight/sum(weights) for weight in weights]
    consensus_prob = 0
    for index in range(len(weights)):
        consensus_prob += weights[index] * probs[index]
    return consensus_prob

def odds_to_decimal_odds(odds):
    if (odds < 0):
        return ((-100 / odds) + 1)
    else:
        return ((odds / 100) + 1)

##Uses Kelly Criterion to calculate optimal bet size
def calc_bet_size(odds, consensus_prob, bankroll):
    decimal_odds = odds_to_decimal_odds(odds)
    frac = consensus_prob - (1-consensus_prob)/(decimal_odds-1)
    if (frac < 0):
        return 0
    else:
        return frac * bankroll

In [2]:
import pandas as pd
import requests
import urllib
from datetime import date

# define API constants
BASE_URL = 'https://api.prop-odds.com'
API_KEY = 'e98LRUF1NW7dFpoJ6wrxzucPq7dj9xnhk7kBZBKnY' # enter your API key here (from https://prop-odds.com/profile)

def get_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

    print('Request failed with status:', response.status_code)
    print(response.reason)
    return {}

In [3]:
def fill_probabilities(odds):
    """Fills the home_probability and away_probability columns in an odds dataframe.
        Parameters:
            odds (dataframe): dataframe containing odds data
        Returns:
            None
    """
    for index, row in odds.iterrows():
        home_prob, away_prob = odds_to_prob2(row['home_team_odds'], row['away_team_odds'])
        odds.loc[index, 'home_probability'] = home_prob
        odds.loc[index, 'away_probability'] = away_prob

In [4]:
# create and load dataframes
todays_games = pd.DataFrame(columns=['home_team', 'away_team', 'prop_odds_game_id'])
todays_odds = pd.DataFrame(columns=['home_team', 'away_team', 'prop_odds_game_id', 'sportsbook', 'home_team_odds', 'away_team_odds', 'home_probability', 'away_probability', 'home_consensus_prob', 'away_consensus_prob', 'home_statistical_edge', 'away_statistical_edge', 'home_exp_value', 'away_exp_value', 'home_bet_size', 'away_bet_size', 'payout'])

historical_games = pd.read_csv('games_full_data.csv').dropna()
historical_odds = historical_games.copy()
historical_odds['home_probability'], historical_odds['away_probability'], historical_odds['home_consensus_prob'], historical_odds['away_consensus_prob'], historical_odds['statistical_edge'], historical_odds['away_statistical_edge'], historical_odds['home_exp_value'], historical_odds['away_exp_value'], historical_odds['home_bet_size'], historical_odds['away_bet_size'], historical_odds['payout'] = None, None, None, None, None, None, None, None, None, None, None

fill_probabilities(historical_odds)


In [5]:
def loss(game):
    """Penalizes the sportsbook based on how wrong their odds were
        Parameters:
            game (series): game data
        Returns:
            float: loss
    """
    home_prob, away_prob = odds_to_prob2(game['home_team_odds'], game['away_team_odds'])

    if (game['winner'] == game['home_team']):
        return 1 - home_prob
    else:
        return 1 - away_prob

In [6]:
def assign_weights_by_loss():
    """Assigns weights to each sportsbook based on historical performance"""
    # assign loss to each game
    historical_games['loss'] = historical_games.apply(loss, axis=1)
    # calculate weights based on total loss for each sportsbook
    average_loss = historical_games.groupby('sportsbook')['loss'].mean()**2
    # weight the sportsbooks inversely to their average loss
    weights = 1 / average_loss
    # normalize weights
    weights = weights / weights.sum()
    return weights

In [7]:
def assign_weights_by_vig():
    """Assigns weights to each sportsbook based on the vig of their odds historically"""
    # calculate average vig of each sportsbook using the vig function, which takes a single pair of odds
    # vig can only take one pair of odds at once, so we use apply to apply it to each row
    vigs = pd.DataFrame(columns = ['sportsbook', 'vig'])
    for index, row in historical_games.iterrows():
        vigs = pd.concat([vigs, pd.DataFrame([[row['sportsbook'], vig(row['home_team_odds'], row['away_team_odds'])]], columns = ['sportsbook', 'vig'])])
    average_vig = vigs.groupby('sportsbook')['vig'].mean()
    # weight the sportsbooks inversely to their average vig
    weights = 1 / average_vig
    # normalize weights
    weights = weights / weights.sum()
    return weights

In [8]:
def get_todays_games():
    """Get today's games from the Prop Odds API"""
    query_params = {
        'date': date.today().strftime('%Y-%m-%d'),
        'tz': 'America/New_York',
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/games/nba?' + params
    return get_request(url)

In [9]:
def fill_todays_games():
    """Fill the todays_games dataframe with today's games"""
    todays_games_json = get_todays_games()
    for game in todays_games_json['games']:
        todays_games.loc[len(todays_games)] = [game['home_team'], game['away_team'], game['game_id']]

In [10]:
def get_game_odds(game_id, market='moneyline'):
    """Returns an object containing the odds provided by the sportsbooks for the given game and market
        Parameters: 
            game_id (str): prop odds game id
            market (str): market to get odds for (moneyline, spread, total)
        Returns:
            odds: dict object containing the odds for the given game and market
    """
    query_params = {
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = BASE_URL + '/beta/odds/' + game_id + '/' + market + '?' + params
    return get_request(url)

In [11]:
def identify_home_away(outcomes, home_team, away_team):
    """Given a list of outcome objects, identifies the home and away outcome objects
        Parameters:
            outcomes (list): list of outcome objects
            home_team (str): home team name
            away_team (str): away team name
        Returns:
            home_outcome: outcome object for the home team
            away_outcome: outcome object for the away team
    """
    home_outcome = None
    away_outcome = None
    home_team_name = home_team.split(' ')[-1]
    away_team_name = away_team.split(' ')[-1]
    for outcome in outcomes:
        if home_team_name in outcome['name']:
            home_outcome = outcome
        elif away_team_name in outcome['name']:
            away_outcome = outcome
    return home_outcome, away_outcome

In [12]:
def add_odds_for_single_game(games, odds, game_id):
    """Adds odds for a single game for all available sportsbooks
        Parameters:
            games (dataframe): dataframe containing game data
            odds (dataframe): dataframe containing odds data
            game_id (str): prop odds game id
        Returns:
            None
    """
    game_odds = get_game_odds(game_id)
    if game_odds:
        for sportsbook in game_odds['sportsbooks']:
            sportsbook_name = sportsbook['bookie_key']
            game = games[games['prop_odds_game_id'] == game_id].copy()
            game['sportsbook'] = sportsbook_name
            home_outcome, away_outcome = identify_home_away(sportsbook['market']['outcomes'], game['home_team'].values[0], game['away_team'].values[0])
            game['home_team_odds'] = home_outcome['odds']
            game['away_team_odds'] = away_outcome['odds']
            game['home_probability'], game['away_probability'] = odds_to_prob2(home_outcome['odds'], away_outcome['odds'])
            game['home_consensus_prob'], game['away_consensus_prob'], game['home_statistical_edge'], game['away_statistical_edge'], game['home_exp_value'], game['away_exp_value'], game['home_bet_size'], game['away_bet_size'], game['payout'] = None, None, None, None, None, None, None, None, None
            odds.loc[len(odds)] = game.values[0]
        